In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn.metrics

In [ ]:
PATH_PROJECT = pathlib.Path(
    "/home/kristian/Projects/kego/pii-detection-removal-from-educational-data/"
)
PATH_COMPETITION = PATH_PROJECT
PATH_TRAIN = PATH_COMPETITION / "train.json"
PATH_SUBMISSION_EXAMPLE = PATH_COMPETITION / "sample_submission.csv"

In [ ]:
!head $PATH_SUBMISSION_EXAMPLE

In [ ]:
train = pd.read_json(PATH_TRAIN)

In [ ]:
train.head()

In [ ]:
def flatten_list(_list):
    return [x for y in _list for x in y]

In [ ]:
unique_labels, unique_labels_n = np.unique(
    flatten_list(train.labels.values), return_counts=True
)

In [ ]:
def create_figure_axes(figure=None, axes=None):
    if figure is None:
        figure = plt.figure()
    if axes is None:
        axes = plt.gca()
    return figure, axes


def plot_histogram(x, df=None, type="string", log_y=False):
    figure, axes = create_figure_axes()
    if isinstance(x, str):
        _x = df[x].values
    else:
        _x = x
    if type == "string":
        unique_labels, unique_labels_n = np.unique(flatten_list(_x), return_counts=True)
        bin_centers = range(len(unique_labels))
        bin_heights = unique_labels_n
        bin_widths = np.diff(bin_centers)[0]
        axes.bar(x=bin_centers, height=bin_heights, width=bin_widths)
        axes.set_xticks(bin_centers, unique_labels, rotation=90)
    if log_y:
        axes.set_yscale("log")

In [ ]:
plot_histogram("labels", train, log_y=True)

In [ ]:
train.head()

In [ ]:
train["document"].nunique(), train.shape

In [ ]:
train

In [ ]:
def get_solutions(train):
    document_n_list = []
    hit_indices_list = []
    hit_tokens_list = []
    hit_types_list = []
    for i in range(train.shape[0]):
        labels = np.array(train.labels.values[i])
        tokens = np.array(train.tokens.values[i])
        document_n = train.document.values[i]
        hit_indices = np.where(labels != "O")[0]
        # print(f"{hit_indices=}")
        hit_types = labels[np.array(hit_indices)]
        hit_tokens = tokens[np.array(hit_indices)]

        document_n_list.append([document_n] * len(hit_indices))
        hit_indices_list.append(hit_indices)
        hit_tokens_list.append(hit_tokens)
        hit_types_list.append(hit_types)
    documents_n = flatten_list(document_n_list)
    hit_index = flatten_list(hit_indices_list)
    hit_tokens = flatten_list(hit_tokens_list)
    hit_type = flatten_list(hit_types_list)
    print(f"{len(documents_n)}, {documents_n=}")
    print(f"{len(hit_index)}, {hit_index=}")
    print(f"{len(hit_type)}, {hit_type=}")
    print(f"{len(hit_tokens)}, {hit_tokens=}")
    # row_id,document,token,label
    solutions = pd.DataFrame(
        data={
            "document": documents_n,
            "token": hit_index,
            "label": hit_type,
            "token name": hit_tokens,
        }
    )
    solutions.index.name = "row_id"
    return solutions


solutions = get_solutions(train)
solutions

In [ ]:
solutions["token name"].nunique()

In [ ]:
solutions.groupby("document").apply(lambda x: x.loc[x["token"].idxmax()])

In [ ]:
!head $PATH_PROJECT"/predictions/test.csv"